In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Fri Jul  1 03:49:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-4kbehc17/farm-haystack_80f54cea708b4e82bb485b3fd76e4a14
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-4kbehc17/farm-haystack_80f54cea708b4e82bb485b3fd76e4a14
  Resolved https://github.com/deepset-ai/haystack.git to commit 1e01cd0efb20d98905c84461e0433d5db4084b7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 kB 6.

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
# Let's first fetch some documents that we want to query

doc_dir = "/content/data1"


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'content': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
# }
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting /content/data1/G.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/F.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/B.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/E.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/D.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/C.txt
INFO - haystack.utils.preprocessing -  Converting /content/data1/A.txt


[<Document: {'content': 'We use different materials and services in our daily life. Some of these are available in our immediate surroundings, while other requirements are met by bringing things from other places. Goods and services do not move from supply locales to demand locales on their own. The movement of these goods and services from their supply locations to demand locations necessitates the need for transport. Some people are engaged in facilitating these movements. These are known to be traders who make the products come to the consumers by transportation. Thus, the pace of development of a country depends upon the production of goods and services as well as their movement over space. Therefore, efficient means of transport are prerequisites for fast development.\nMovement of these goods and services can be over three important domains of our earth i.e. land, water and air. Based on these, transport can also be classified into land, water and air transport.\nFor a long time, 

In [ ]:
from haystack.nodes import ElasticsearchRetriever
#retreiving the document from document store using ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

WARNING - haystack.nodes.retriever.sparse -  This class is now deprecated. Please use the BM25Retriever instead


In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
def test(text):
  prediction = pipe.run(
    query=text, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}})
  print(prediction)
  return prediction

#def test(text):
    #prediction = pipe.run(
    #query=text, params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 1}})
    #print(prediction)
    #return prediction

In [ ]:
test("What did the spread of print culture in nineteenth century India mean to?")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.87 Batches/s]

{'query': 'What did the spread of print culture in nineteenth century India mean to?', 'no_ans_gap': -4.342446565628052, 'answers': [<Answer {'answer': 'resource conservation', 'type': 'extractive', 'score': 0.09897268563508987, 'context': 'ept of ‘Sustainable Development’ and advocated it as a means for resource conservation, which was subsequently published in a book entitled Our Common', 'offsets_in_document': [{'start': 11502, 'end': 11523}], 'offsets_in_context': [{'start': 65, 'end': 86}], 'document_id': '74803ab4f6aa16f121026e2b58f452f8', 'meta': {'name': 'A.txt'}}>], 'documents': [<Document: {'content': 'We share this planet with millions of other living beings, starting from micro-organisms and bacteria, lichens to banyan trees, elephants and blue whales. This entire habitat that we live in has immense biodiversity. We humans along with all living organisms form a complex web of ecological system in which we are only a part and very much dependent on this system for our own ex

{'answers': [<Answer {'answer': 'resource conservation', 'type': 'extractive', 'score': 0.09897268563508987, 'context': 'ept of ‘Sustainable Development’ and advocated it as a means for resource conservation, which was subsequently published in a book entitled Our Common', 'offsets_in_document': [{'start': 11502, 'end': 11523}], 'offsets_in_context': [{'start': 65, 'end': 86}], 'document_id': '74803ab4f6aa16f121026e2b58f452f8', 'meta': {'name': 'A.txt'}}>],
 'documents': [<Document: {'content': 'We share this planet with millions of other living beings, starting from micro-organisms and bacteria, lichens to banyan trees, elephants and blue whales. This entire habitat that we live in has immense biodiversity. We humans along with all living organisms form a complex web of ecological system in which we are only a part and very much dependent on this system for our own existence. For example, the plants, animals and micro-organisms re-create the quality of the air we breathe, the water we

In [ ]:
! pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, render_template, send_file, jsonify, Response
from flask_ngrok import run_with_ngrok

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/",methods=["POST"])
def abc():
    data= request.get_json(force=True)
    print(data)
    text = data['query']
    result = test(text)
    # print("**** result:::: ", result)
    res = result.get('answers')
    data={"answers":res[0].to_dict().get('answer')}
    #data = {"answers":res["answers"]}
    # print(data)
    return jsonify(data)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO - werkzeug -   * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://68d3-34-72-4-14.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{'query': 'what is border road organization?'}


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.57 Batches/s]
INFO - werkzeug -  127.0.0.1 - - [01/Jul/2022 04:30:20] "POST / HTTP/1.1" 200 -


{'query': 'what is border road organization?', 'no_ans_gap': 7.349087953567505, 'answers': [<Answer {'answer': 'Government of India undertaking constructs and maintains roads in the bordering areas of the country', 'type': 'extractive', 'score': 0.46578896045684814, 'context': 'der Roads Organisation a Government of India undertaking constructs and maintains roads in the bordering areas of the country. This organisation was e', 'offsets_in_document': [{'start': 4816, 'end': 4916}], 'offsets_in_context': [{'start': 25, 'end': 125}], 'document_id': 'f9444f7216b9de5adc43521e4a266441', 'meta': {'name': 'G.txt'}}>], 'documents': [<Document: {'content': 'We use different materials and services in our daily life. Some of these are available in our immediate surroundings, while other requirements are met by bringing things from other places. Goods and services do not move from supply locales to demand locales on their own. The movement of these goods and services from their supply locations to

...

{'query': 'When was the first meeting held', 'no_ans_gap': 3.492816925048828, 'answers': [<Answer {'answer': '1968', 'type': 'extractive', 'score': 0.7930925190448761, 'context': 'ted resource conservation for the first time in a more systematic way in 1968. Subsequently, in 1974, Gandhian philosophy was once again presented by ', 'offsets_in_document': [{'start': 11151, 'end': 11155}], 'offsets_in_context': [{'start': 73, 'end': 77}], 'document_id': '74803ab4f6aa16f121026e2b58f452f8', 'meta': {'name': 'A.txt'}}>], 'documents': [<Document: {'content': 'You have studied that the earth’s crust is made up of different minerals embedded in the rocks. Various metals are extracted from these minerals after proper refinement.\nMinerals are an indispensable part of our lives. Almost everything we use, from a tiny pin to a towering building or a big ship, all are made from minerals. The railway lines and the tarmac (paving) of the roads, our implements and machinery too are made from minerals. 

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.38 Batches/s]
INFO - werkzeug -  127.0.0.1 - - [01/Jul/2022 05:36:10] "POST / HTTP/1.1" 200 -


{'query': 'Jute?', 'no_ans_gap': -2.6535056233406067, 'answers': [<Answer {'answer': 'jhumming', 'type': 'extractive', 'score': 0.173268660902977, 'context': ' parts of the country.\nCan you name some such types of farmings?\nIt is jhumming in north-eastern states like Assam, Meghalaya, Mizoram and Nagaland; P', 'offsets_in_document': [{'start': 1747, 'end': 1755}], 'offsets_in_context': [{'start': 71, 'end': 79}], 'document_id': '6ee99d8fab34809c23726c84574f677a', 'meta': {'name': 'D.txt'}}>], 'documents': [<Document: {'content': 'Production of goods in large quantities after processing from raw materials to more valuable products is called manufacturing. Do you know that paper is manufactured from wood, sugar from sugarcane, iron and steel from iron ore and aluminium from bauxite? Do you also know that some types of clothes are manufactured from yarn which itself is an industrial product?\nPeople employed in the secondary activities manufacture the primary materials into finished goo

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.97 Batches/s]
INFO - werkzeug -  127.0.0.1 - - [01/Jul/2022 05:36:54] "POST / HTTP/1.1" 200 -


{'query': 'What is jute', 'no_ans_gap': 7.261043071746826, 'answers': [<Answer {'answer': 'Cotton', 'type': 'extractive', 'score': 0.625570148229599, 'context': ' materials and produce light goods such as electrical goods industries.\nCotton, jute, silk, woollen textiles, sugar and edible oil, etc. industries ar', 'offsets_in_document': [{'start': 6988, 'end': 6994}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': 'e95a1e4670a0ae941c2fac9ab0a7069d', 'meta': {'name': 'F.txt'}}>], 'documents': [<Document: {'content': 'Production of goods in large quantities after processing from raw materials to more valuable products is called manufacturing. Do you know that paper is manufactured from wood, sugar from sugarcane, iron and steel from iron ore and aluminium from bauxite? Do you also know that some types of clothes are manufactured from yarn which itself is an industrial product?\nPeople employed in the secondary activities manufacture the primary materials into finished g



---



---



In [ ]:
result = test("Why Indians were outraged by the Rowlatt Act?")

In [ ]:
res = result.get('answers')

In [ ]:
res = res[0].to_dict()

In [ ]:
print(res.get('context'))